In [73]:
import numpy as np
import tensorflow as tf

print(tf.__version__)

2.11.0


In [74]:

from pathlib import Path

local_dir = Path().cwd()
train_dir = local_dir / 'data/train/'
test_dir = local_dir / 'data/test/'
val_dir = local_dir / 'data/val/'

#Reads the data from disk
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir, image_size=(224, 224), seed=123)
val_ds = tf.keras.utils.image_dataset_from_directory(val_dir, image_size=(224, 224), seed=123)
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir, image_size=(224, 224), seed=123)

print(train_ds.class_names)
print(val_ds.class_names)
print(test_ds.class_names)



Found 3045 files belonging to 2 classes.
Found 16 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
['NORMAL', 'PNEUMONIA']
['NORMAL', 'PNEUMONIA']
['NORMAL', 'PNEUMONIA']


In [75]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical", seed=123),
    tf.keras.layers.RandomRotation(0.2)
])

In [76]:
# prints the shape of the training dataset
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 224, 224, 3)
(32,)


In [77]:
# prints the shape of the training dataset
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 224, 224, 3)
(32,)


In [78]:
# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds_norm = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds_norm = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [79]:
# Building the network
NUM_CLASSES = 2

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(64, 3, strides=1, padding='same', activation='relu'),
  tf.keras.layers.MaxPool2D(2, strides=2),
  tf.keras.layers.Conv2D(128, 3, strides=1, padding='same', activation='relu'),
  tf.keras.layers.MaxPool2D(2, strides=2),
  tf.keras.layers.Conv2D(256, 3, strides=1, padding='same', activation='relu'),
  tf.keras.layers.MaxPool2D(2, strides=2),
  tf.keras.layers.Conv2D(512, 3, strides=1, padding='same', activation='relu'),
  tf.keras.layers.MaxPool2D(2, strides=2),
  tf.keras.layers.Conv2D(512, 3, strides=1, padding='same', activation='relu'),
  tf.keras.layers.MaxPool2D(2, strides=2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(NUM_CLASSES)
])

In [80]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [81]:
# Training the network
model.fit(
  train_ds,
  validation_data=val_ds,
  batch_size=64,
  epochs=40
)

Epoch 1/40
96/96 [==============================] - 394s 4s/step - loss: 0.4997 - accuracy: 0.7911 - val_loss: 0.3371 - val_accuracy: 0.8125
Epoch 2/40
96/96 [==============================] - 407s 4s/step - loss: 0.1767 - accuracy: 0.9333 - val_loss: 0.3214 - val_accuracy: 0.9375
Epoch 3/40
96/96 [==============================] - 397s 4s/step - loss: 0.1200 - accuracy: 0.9580 - val_loss: 0.2221 - val_accuracy: 0.8750
Epoch 4/40
96/96 [==============================] - 393s 4s/step - loss: 0.0974 - accuracy: 0.9681 - val_loss: 0.3601 - val_accuracy: 0.8750
Epoch 5/40
96/96 [==============================] - 384s 4s/step - loss: 0.0764 - accuracy: 0.9744 - val_loss: 0.1847 - val_accuracy: 0.9375
Epoch 6/40
96/96 [==============================] - 374s 4s/step - loss: 0.0723 - accuracy: 0.9747 - val_loss: 0.2321 - val_accuracy: 0.8750
Epoch 7/40
96/96 [==============================] - 360s 4s/step - loss: 0.0652 - accuracy: 0.9741 - val_loss: 0.0832 - val_accuracy: 1.0000
Epoch 8/40
96

In [82]:
score = model.evaluate(test_ds)

20/20 [==============================] - 20s 957ms/step - loss: 4.5050 - accuracy: 0.8013


In [83]:

print(score)

[4.505031585693359, 0.8012820482254028]


In [84]:
model.save('tfmodel')

INFO:tensorflow:Assets written to: tfmodel\assets


INFO:tensorflow:Assets written to: tfmodel\assets
